In [1]:
import os
import docker

docker_client = docker.from_env()

In [25]:
# Run a mongo 5 container on the port 27015
mongo5_container = docker_client.containers.run(
    'mongo:5',
    detach=True,
    name='mongo-5',
    #remove=True,
    ports={'27017/tcp': 27015},
    volumes={
        os.path.join(os.getcwd(), 'db5'): {'bind': '/data/db', 'mode': 'rw'}
    }
)

In [5]:
# Run a mongo 4 container on the port 27014
mongo4_container = docker_client.containers.run(
    'mongo:4',
    detach=True,
    name='mongo-4',
    remove=True,
    ports={'27017/tcp': 27014},
    volumes={
        os.path.join(os.getcwd(), 'db4'): {'bind': '/data/db', 'mode': 'rw'}
    }
)

In [18]:
import json

# Get sample-data
with open('./data_1.json') as file:
    data_1 = json.load(file)
    
with open('./data_2.json') as file:
    data_2 = json.load(file)
    
from pymongo import MongoClient

# Import sample data in mongo
client5 = MongoClient('mongodb://localhost:27015/')
client4 = MongoClient('mongodb://localhost:27014/')

client4.sample_data['data_1'].drop()
client4.sample_data['data_1'].insert_many(data_1)
client4.sample_data['data_2'].drop()
client4.sample_data['data_2'].insert_many(data_2)

client5.sample_data['data_1'].drop()
client5.sample_data['data_1'].insert_many(data_1)
client5.sample_data['data_2'].drop()
client5.sample_data['data_2'].insert_many(data_2)

In [19]:
# Execute problematic query
with open('./minimal-query.json') as file:
    query = json.load(file)

# It works with mongo4...
list(client4.sample_data['data_1'].aggregate(query))

[{'_id': ObjectId('61656210086b742b1a76604f'),
  'Identifier': 'K4',
  '_lookedUp': [{'data_2': [{'_id': {'Identifier': 'K4'}}]}]},
 {'_id': ObjectId('61656210086b742b1a766050'),
  'Identifier': 'K8',
  '_lookedUp': [{'data_2': [{'_id': {'Identifier': 'K4'}}]}]}]

In [27]:
# But fails with mongo5...
list(client5.sample_data['data_1'].aggregate(query))

KeyboardInterrupt: 

In [31]:
# logs of the container (change the indexes to navigate)

[(
    json.loads(line)['ctx'],
    json.loads(line).get('attr', {}).get('message', None) or json.loads(line).get('attr', {}).get('frame', {}).get('s', None)
 ) for line in mongo5_container.logs().decode().split('\n')[-55:-1]
]


[('conn5', None),
 ('conn5', 'Invalid access at address: 0x79550000'),
 ('conn5', 'Got signal: 11 (Segmentation fault).\n'),
 ('conn5', None),
 ('conn5', None),
 ('conn5',
  '_ZN5mongo18stack_trace_detail12_GLOBAL__N_119printStackTraceImplERKNS1_7OptionsEPNS_14StackTraceSinkE.constprop.357'),
 ('conn5', '_ZN5mongo15printStackTraceEv'),
 ('conn5', 'abruptQuitWithAddrSignal'),
 ('conn5', 'funlockfile'),
 ('conn5', None),
 ('conn5', '_ZN5mongo25DocumentSourceTeeConsumer9doDisposeEv'),
 ('conn5', '_ZN5mongo19DocumentSourceFacetD0Ev'),
 ('conn5',
  '_ZN5mongo37DocumentSourceSequentialDocumentCache12doOptimizeAtESt14_List_iteratorIN5boost13intrusive_ptrINS_14DocumentSourceEEEEPNSt7__cxx114listIS5_SaIS5_EEE'),
 ('conn5',
  '_ZN5mongo8Pipeline17optimizeContainerEPNSt7__cxx114listIN5boost13intrusive_ptrINS_14DocumentSourceEEESaIS6_EEE'),
 ('conn5', '_ZN5mongo8Pipeline16optimizePipelineEv'),
 ('conn5', '_ZN5mongo20DocumentSourceLookUp13buildPipelineERKNS_8DocumentE'),
 ('conn5', '_ZN5mongo20Docu